<a href="https://colab.research.google.com/github/JanPK63/ChatGPT_Trading_Bot/blob/main/trade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install ccxt
pip install numpy
pip install time
pip install threading
pip install tkinter

import ccxt
import numpy as np
import time
import threading
import tkinter as tk
from tkinter import ttk, scrolledtext

# Setup CCXT
bybit = ccxt.bybit({
    'apiKey': 'YOUR_API_KEY',
    'secret': 'YOUR_SECRET_KEY',
})

# Globale variabelen
entry_price = None
stop_loss_price = None
trailing_profit_set = False
running = False
leverage = 5
order_percentage = 0.01

def simple_moving_average(data, period):
    return np.convolve(data, np.ones(period)/period, mode='valid')

def trading_logic():
    global entry_price, stop_loss_price, trailing_profit_set, running, leverage, order_percentage

    trailing_profit_distance = 0.001

    while running:
        try:
            # Verzamel marktgegevens
            ohlc = bybit.fetch_ohlcv('BTC/USDT', '1h')
            closes = [x[4] for x in ohlc]
            sma = simple_moving_average(closes, 10)

            latest_data = np.array(closes[-30:])
            prediction = clf.predict(latest_data)
            sma_signal = closes[-1] > sma[-1]

            buy_signal = prediction[0] == 1 and sma_signal
            sell_signal = prediction[0] == 0 or not sma_signal

            # Stel hefboomwerking in
            bybit.set_leverage(symbol='BTC/USDT', leverage=leverage)

            if buy_signal and entry_price is None:
                available_balance = bybit.fetch_balance()['BTC']['free']
                order_size = order_percentage * available_balance

                order = bybit.create_market_buy_order('BTC/USDT', order_size)
                entry_price = order['price']
                stop_loss_price = entry_price * 0.99
                log_text.insert(tk.END, f"Bought at {entry_price}\n")

            elif sell_signal and entry_price is not None:
                current_price = bybit.fetch_ticker('BTC/USDT')['last']

                if not trailing_profit_set and current_price > entry_price * 1.001:
                    stop_loss_price = current_price - (entry_price * trailing_profit_distance)
                    trailing_profit_set = True
                elif trailing_profit_set and current_price - stop_loss_price > entry_price * trailing_profit_distance:
                    stop_loss_price = current_price - (entry_price * trailing_profit_distance)

                if current_price <= stop_loss_price:
                    order_size = order_percentage * bybit.fetch_balance()['BTC']['free']
                    bybit.create_market_sell_order('BTC/USDT', order_size)
                    log_text.insert(tk.END, f"Sold at {current_price}\n")
                    entry_price = None
                    stop_loss_price = None
                    trailing_profit_set = False

            time.sleep(3600)

        except Exception as e:
            log_text.insert(tk.END, f"Error: {e}\n")
            time.sleep(60)  # Wacht een minuut na een fout

def start_bot():
    global running
    running = True
    threading.Thread(target=trading_logic).start()

def stop_bot():
    global running
    running = False

def update_parameters():
    global leverage, order_percentage
    leverage = float(leverage_input.get())
    order_percentage = float(order_percent_input.get())/100

# GUI opzet
root = tk.Tk()
root.title("Trading Bot")

frame = ttk.Frame(root)
frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

# Start en Stop knoppen
start_button = ttk.Button(frame, text="Start Bot", command=start_bot)
start_button.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

stop_button = ttk.Button(frame, text="Stop Bot", command=stop_bot)
stop_button.grid(row=0, column=1, sticky=(tk.W, tk.E, tk.N, tk.S))

# Parameters
leverage_label = ttk.Label(frame, text="Leverage:")
leverage_label.grid(row=1, column=0, sticky=tk.W)
leverage_input = ttk.Entry(frame)
leverage_input.grid(row=1, column=1, sticky=tk.W)
leverage_input.insert(0, "5")

order_percent_label = ttk.Label(frame, text="Order Percentage:")
order_percent_label.grid(row=2, column=0, sticky=tk.W)
order_percent_input = ttk.Entry(frame)
order_percent_input.grid(row=2, column=1, sticky=tk.W)
order_percent_input.insert(0, "1")

update_button = ttk.Button(frame, text="Update Parameters", command=update_parameters)
update_button.grid(row=3, column=0, columnspan=2, sticky=(tk.W, tk.E, tk.N, tk.S))

# Logs
log_text = scrolledtext.ScrolledText(frame, width=70, height=10)
log_text.grid(row=4, column=0, columnspan=2, sticky=(tk.W, tk.E, tk.N, tk.S))

root.mainloop()



SyntaxError: ignored